In [17]:
### WIDTH = 10;

FP16_MAN_LEN = 10
FP16_EXP_LEN = 5
FP16_EXP_BIAS = 2**(FP16_EXP_LEN-1)-1         # 15
FP32_MAN_LEN = 23
FP32_EXP_LEN = 8
FP32_EXP_BIAS = 2**(FP32_EXP_LEN-1)-1         # 127
FP29i_MAN_LEN = 22
FP29i_EXP_LEN = 6
FP29i_EXP_BIAS = 2**(FP29i_EXP_LEN-1)-1         # 127
FP64_MAN_LEN = 52
FP64_EXP_LEN = 11
FP64_EXP_BIAS = 2**(FP64_EXP_LEN-1)-1         # 1023

FMT_FP16  = [1,FP16_MAN_LEN,FP16_EXP_BIAS,FP16_EXP_LEN]
FMT_FP32  = [1,FP32_MAN_LEN,FP32_EXP_BIAS,FP32_EXP_LEN]
FMT_FP32N = [3,FP32_MAN_LEN,FP32_EXP_BIAS,FP32_EXP_LEN]
FMT_FP29i = [1,FP29i_MAN_LEN,FP29i_EXP_BIAS,FP29i_EXP_LEN]
#            \._ 1: has_denorm 3: force_denorm 

# CAUTION: Python List and Binary indexes left-to-right
# Call list reverse "a.reverse()"" before indexing in verilog fashion

# Useful Lambda Functions

# Convert Binary String (without '0b') to Binary Array
strdnum  = lambda s    : list(int(x) for x in s)
# Convert Binary Array to Binary String (without '0b')
numdstr  = lambda n    : "".join(str(x) for x in n)
# Convert Binary Array to Integer
bin2int  = lambda b    : int(numdstr(b),2)
# Convert Integer to Binary Array
int2bin  = lambda i, w : strdnum(bin(i)[2:].zfill(w))
# Reverse Indexing
indexrev = lambda a, i : a[len(a)-1-i]
# Check If an Array Only Has One Variety of Element
is_all   = lambda a, e : (a.count(e)==len(a))

# BINARY TOOLS
xor      = lambda a, b       : int((a and not b) or (not a and b))
# Word size is strictly defined by the input vectors +1
binadd   = lambda ba, bb, bci: int2bin(bin2int(ba)+bin2int(bb)+bci,max(len(ba),len(bb))+1)
# Word size is strictly defined by the input vectors
def binaddt(ba, bb, bci):
    ws = max(len(ba),len(bb));
    k = int2bin(bin2int(ba)+bin2int(bb)+bci,ws)
    return k[len(k)-ws:]
# Sign Extension
binsext  = lambda b, ws      : [b[0]]*(ws-len(b))+b;
# Negate Binary Value; word size unchanged
binneg   = lambda b          : binadd(list(1-i for i in b),[0],1)[1:len(b)+1];

# Convert binary array to Python float value
def fpdecode(fmt,pack):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    sgn = pack[0]
    exp = pack[1:exp_len+1]
    man = pack[exp_len+1:]
    
    if (is_all(exp,0) and has_denorm) or force_denorm:
        # DENORM
        print("fpdecode: DENORM")
        t = float(bin2int(man))*2**(-man_len+1)
        t = t*(2**(-exp_bias))
    
    # SPECIAL numbers are not dealt-with in this implementation
    elif (is_all(exp,1)) :
        print("fpdecode: SPECIAL NUMBER");
        return float("nan")
    else:
        # NORM
        t = float(bin2int([1] + man))*2**(-man_len)
        t = t*(2**(float(bin2int(exp))-exp_bias))
        
    if sgn == 1:
            t = t*-1
    return t

# Conver float value to binary array
import struct
from math import log
def double2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!d', num))

def half2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!e', num))

# Convert Python float (double) value to binary array in fmt
# FP16 doesn't need this function:
#    just call half2binary to get a binary string, 
#    and then convert it to a binary array with strdnum(s)
def fpencode(fmt,num):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    a = double2binary(num);
    sgn = a[0]
    exp64 = a[1:FP64_EXP_LEN+1]
    man64 = a[FP64_EXP_LEN+1:]
    
    # Convert Exponent
    exp64i = bin2int(exp64)
    exp = exp64i - FP64_EXP_BIAS + exp_bias
    if is_all(exp64,'0'):
        # Zeros
        return [int(sgn)] + [0]*exp_len + [0]*man_len
    if is_all(exp64,'1'):
        # Infinity
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if abs(exp) >= 2**(exp_len)-1:
        # Overflow, not implemented yet
        print("ERROR: EXP OVERFLOW");
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if(exp>0):
        exp = int2bin(exp,FP64_EXP_LEN)[FP64_EXP_LEN - exp_len:] 
        # Convert Mantissa
        man = strdnum(man64[0:man_len])
    else:
        # Denorm
        man = ([0]*abs(exp)+[1]+strdnum(man64))[0:man_len]
        exp = int2bin(0,exp_len)
    return [int(sgn)] + exp + man;


man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,1]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [1,1,1,1,1,1,1,1,1,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,0]
pack = [1] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

6.103515625e-05
fpdecode: DENORM
6.097555160522461e-05
fpdecode: DENORM
5.960464477539063e-08
fpdecode: DENORM
-0.0


In [13]:
# FP16 Back-and-forth Conversion Test
# https://en.wikipedia.org/wiki/Half-precision_floating-point_format

def fp16verify(s, desc):
    t = fpdecode(FMT_FP16,strdnum(s));
    # print(t);
    k = numdstr(fpencode(FMT_FP16,t));
    if k==s:
        print('- pass -', desc)
    else:
        print('- FAIL - ', desc)
        print('Expected:\t', s);
    print('Result  :\t', k);

fp16verify('0000000000000001','Smallest Positive Subnormal')
fp16verify('0000001010101010','Subnormal')
fp16verify('0000001111111111','Largest Subnormal')
fp16verify('0000010000000000','Smallest Positive Normal')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0011101111111111','Largest Number Less Than One')
fp16verify('0011110000000000','One')
fp16verify('0011110000000001','Smallest Number Larger Than One')
fp16verify('0111101111111111','Largest Normal Number')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0111110000000000','Infinity')
fp16verify('1000000000000000','Negative Zero')
fp16verify('0000000000000000','Positive Zero')
fp16verify('1100000000000000','-2')
fp16verify('1111111111111111','-Infinity')

fpdecode: DENORM
- pass - Smallest Positive Subnormal
Result  :	 0000000000000001
fpdecode: DENORM
- pass - Subnormal
Result  :	 0000001010101010
fpdecode: DENORM
- pass - Largest Subnormal
Result  :	 0000001111111111
- pass - Smallest Positive Normal
Result  :	 0000010000000000
- pass - Nearest Value to 1/3
Result  :	 0011010101010101
- pass - Largest Number Less Than One
Result  :	 0011101111111111
- pass - One
Result  :	 0011110000000000
- pass - Smallest Number Larger Than One
Result  :	 0011110000000001
- pass - Largest Normal Number
Result  :	 0111101111111111
- pass - Nearest Value to 1/3
Result  :	 0011010101010101
fpdecode: SPECIAL NUMBER
- pass - Infinity
Result  :	 0111110000000000
fpdecode: DENORM
- pass - Negative Zero
Result  :	 1000000000000000
fpdecode: DENORM
- pass - Positive Zero
Result  :	 0000000000000000
- pass - -2
Result  :	 1100000000000000
fpdecode: SPECIAL NUMBER
- FAIL -  -Infinity
Expected:	 1111111111111111
Result  :	 0111110000000000


In [476]:
fpencode(FMT_FP16,1+30.76e-3)

[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [14]:
# Note: For simplicity, ALL INTERNAL DATA is encoded in NON-NORMALIZED FORMAT
# that is: no hidden ones at the beginning of each encoded number, all digits are explicitly listed

# Show Verbose Debug Messages
DEBUGMODE = 0;

# This block locates at the beginning of the FIR pipeline
# Convert Normal FP16 Numbers to Internal Non-Normalized FMT (IFMT)
def fp16_to_ifmt(fmt, fp16):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Convert and unpack
    pack = strdnum(half2binary(fp16))
    sgn = pack[0]
    exp = pack[1:FP16_EXP_LEN+1]
    man = pack[FP16_EXP_LEN+1:]

    # Add the new bias value
    expn = bin2int(exp) - FP16_EXP_BIAS + exp_bias

    # Deal with Denorm
    if(is_all(exp,0)):
        if(DEBUGMODE):
            print("DENORM");
        man = man + [0]*(man_len - FP16_MAN_LEN)
    else:
        # This causes problem when man_len < FP16_MAX_LEN,
        # which shouldn't happen in this project, ignore that
        man = [1] + man + [0]*(man_len - FP16_MAN_LEN - 1)
    
    # Save new EXP
    exp = int2bin(expn, exp_len)
    if(DEBUGMODE):
        print(sgn, numdstr(exp), numdstr(man))
        
    return [int(sgn)] + exp + man

In [16]:
# Show Verbose Debug Messages
DEBUGMODE = 0;

# Implementation of the Floating Point Adder
# Internal Format Only (no hidden 1)
# fmt:   Floating Point Format (Defined in the first block)
# packa: Addend 1              (binary array format, MSB as [0])
# packa: Addend 2              (binary array format, MSB as [0])
def fpadd_ifmt(fmt, packa, packb):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Unpack Data
    # Caution: Here we're assuming that all incoming data are in denorm format
    # They have exponents (unlike IEEE terminalogy), but also have an explicit 1
    sgna = packa[0]
    expa = packa[1:exp_len+1]
    mana = [0]*2 + packa[exp_len+1:]
    
    sgnb = packb[0]
    expb = packb[1:exp_len+1]
    manb = [0]*2 + packb[exp_len+1:]
    
    if(DEBUGMODE):
        print("A: ", sgna, numdstr(expa), numdstr(mana))
        print("B: ", sgnb, numdstr(expb), numdstr(manb))
    
    # - PIPELINE STEP 1: Exponent Compare 
    expa_sub_b = bin2int(expa) - bin2int(expb)
    expa_gt_b = (expa_sub_b > 0)
    
    # Addsub Control Signals:
    #          ea > eb     ea < eb
    #            SRB         SRA
    # sa sb  sy    my     sy    my
    #  0  0   0  ma + mb   0  mb + ma
    #  0  1   0  ma - mb   1  mb - ma
    #  1  0   1  ma - mb   0  mb - ma
    #  1  1   1  ma + mb   1  mb + ma
    
    ctrl_manxch = -expa_gt_b         # Exchange ALU Inputs
    ctrl_mansub = xor(sgna,sgnb)     # Subtract Mode
    if(expa_gt_b):
        sgny = sgna;
    else:
        sgny = sgnb;
        
    # - PIPELINE STEP 2: Shift
    # Shifter
    if(expa_gt_b):
        expy = expa
        manb = ([0]*(abs(expa_sub_b)) + manb)[0:len(manb)];
    else:
        expy = expb
        mana = ([0]*(abs(expa_sub_b)) + mana)[0:len(mana)];
    
    # - PIPELINE STEP 3: Addsub
    # ALU MUX
    if expa_gt_b:
        alua = mana
        alub = manb
    else:
        alua = manb
        alub = mana

    if ctrl_mansub:
        alub = binneg(alub)
    many = binadd(alua,alub,0);
    many = many[1:]
    if(DEBUGMODE):
        print("ALU:")
        print("A:", numdstr(alua))
        print("B:", numdstr(alub))
        print("after ALU: exp:",numdstr(expy),"many:", numdstr(many));
    
    # - PIPELINE STEP 4: Normalize - detect
    if(is_all(many,0)):
        # Zero
        return [0] + [0]*exp_len + [0]*man_len
    
    # Find the initial one
    many_zerocount = len(many) - len(numdstr(many).lstrip('0'));
    
    # - PIPELINE STEP 5: Normalize - Shift
    
    many = (many+[0]*(many_zerocount+1))[many_zerocount:]
    expyn = bin2int(expy)-many_zerocount
    expy = int2bin(expyn,len(expy))
    if(DEBUGMODE):
        print("many_zerocount:", many_zerocount)
        print("after SFT: exp:",numdstr(expy),"many:", numdstr(many));
    
    # Deal with denorm
    if(is_all(expy,0)):
        if(DEBUGMODE):
            print("Denorm")
        return [int(sgny)] + [0]*exp_len + many[0:man_len]
    else:
        a = numdstr([int(sgny)] + expy + many[1:]);
        if(DEBUGMODE):
            print(a[0],a[1:FP32_EXP_LEN+1],a[FP32_EXP_LEN+1:])
        return [int(sgny)] + expy + many[1:]

# - Deprecated -
def fpadd_verif(fpa, fpb, msg):
    fpy = fpa + fpb
    fpa = fp16_to_ifmt(FMT_FP32N,fpa)
    fpb = fp16_to_ifmt(FMT_FP32N,fpb)
    k = fpdecode(FMT_FP32, fpadd_ifmt(FMT_FP32N, fpa, fpb));
    if(k==fpy):
        print('- pass -', msg)
    else:
        print('- FAIL - ', msg)
    print("Expected:  ", fpy);
    print("Actual:    ", k)

fpadd_verif(-1,1,'Unity')
fpadd_verif(2,0,'Add Positive With Zero')
fpadd_verif(-2,0,'Add Negative With Zero')
fpadd_verif(-2,1/3,'Infinite Fractal')

# KNOWN BUG:
# The comparison mostly not work because the internal fp format is always double
# whose precision is much higher than fp16,
# Such comparison can fail due to the lack of precision
# I should find a better way to compare

fpdecode: DENORM
- pass - Unity
Expected:   0
Actual:     0.0
- pass - Add Positive With Zero
Expected:   2
Actual:     2.0
- pass - Add Negative With Zero
Expected:   -2
Actual:     -2.0
- FAIL -  Infinite Fractal
Expected:   -1.6666666666666667
Actual:     -1.666748046875


In [2]:
# Implementation of a Radix-4 Booth Mutiplier
# This is essentially the text-book version of this multiplier
# No hard-ware level optimization is done yet

import functools
listcmp = lambda l1, l2: functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,l1,l2), True)

DEBUGMODE = 0;

# Convert Multiplicant to Booth Recording
# man: Multiplicant (binary array format, MSB as [0])
def boothr4_encode(man):
    man = binsext(man + [0],len(man)+3)
    brlen = len(man)//2
    br = [0]*brlen
    for i in range(brlen):
        ir = len(man)-1-(2*i)
        u = (man[ir-2],man[ir-1],man[ir])
        if(u==(0,0,0)):
            a = 0
        elif(u==(0,0,1)):
            a = 1
        elif(u==(0,1,0)):
            a = 1
        elif(u==(0,1,1)):
            a = 2
        elif(u==(1,0,0)):
            a = -2
        elif(u==(1,0,1)):
            a = -1
        elif(u==(1,1,0)):
            a = -1
        elif(u==(1,1,1)):
            a = 0
        else:
            print("boothr4_encode: ERROR: UNEXPECTED INPUT")
        #br[brlen-i-1]=a;
        br[i]=a
    return br

# Convert booth recording notation to binary chain
# br:  Booth Recorded Multiplier (integer array format, LSB as [0])
def boothr4_br2bin(br):
    bbin = []
    for i in br:
        if(i==0):
            k = [0,0,0]
        if(i==1):
            k = [0,0,1]
        if(i==2):
            k = [0,1,0]
        if(i==-1):
            k = [1,1,1]
        if(i==-2):
            k = [1,1,0]
        bbin = bbin + k;
    return bbin

# Booth Radix-4, shift and multiply
# br:  Booth Recorded Multiplier (integer array format, LSB as [0])
# num: Multiplicant              (binary array format, MSB as [0])
def boothr4_slmult(br, num):
    acc = [0]*(2*len(num))
    for i in br:
        if(i==1):
            k = num;
        elif(i==2):
            k = num+[0];
        elif(i==-1):
            k = binneg(num);
        elif(i==-2):
            k = binneg(num+[0]);
        elif(i==0):
            k = [0]
        else:
            print("boothr4_slmult: ERROR: UNEXPECTED BR INPUT")
        k = binsext(k,len(acc))
        if(DEBUGMODE):
            print("i:",i);
            print("A:",numdstr(acc));
            print("K:",numdstr(k));
        acc = binaddt(acc,k,0)
        num = num+[0]*2;
    return acc

# Verify Booth Radix-4 Multiplier Operation
# ia: Multiplier   (integer)
# ib: Multiplicand (integer)
# ws: Wordsize     (integer)
# msg: Message     (char string)
def boothr4_verif(ia,ib,ws,msg):
    iy=ia*ib
    if(ia>2**(ws-1)-1) or (ia<-(2**(ws-1))) :
        print("boothr4_verif: ERROR: IA OVERFLOW")
    if(ib>2**(ws-1)-1) or (ib<-(2**(ws-1))) :
        print("boothr4_verif: ERROR: IB OVERFLOW")
    ba = int2bin(abs(ia), ws)
    bb = int2bin(abs(ib), ws)
    by = int2bin(abs(iy), 2*ws)
    if(ia<0):
        ba = binneg(ba);
    if(ib<0):
        bb = binneg(bb);    
    if(iy<0):
        by = binneg(by);

    bbra = boothr4_encode(ba)
    bbry = numdstr(boothr4_slmult(bbra, bb));
    if(DEBUGMODE):
        print("A:      ",ia)
        print("B:      ",ib)
        print("A (br4,org):",bbra)
        print("A (br4,bin):",numdstr(boothr4_br2bin(bbra)))
        print("Booth Y:",bin2int(bbry))
        print("Ideal Y:",iy)
    if(bin2int(by)==bin2int(bbry)):
        print(" -pass- ",ia,"x",ib,"=",iy,msg);
    else:
        print(" -FAIL- ",ia,"x",ib,"=",iy,msg);
        print("Expected:",numdstr(by),bin2int(by))
        print("Actual:  ",numdstr(bbry),bin2int(bbry))
        
boothr4_verif(28,6,8,"")
boothr4_verif(28,32,8,"")
boothr4_verif(-100,6,8,"")
boothr4_verif(0,127,8,"")
boothr4_verif(-1,1,8,"")
boothr4_verif(-128,127,8,"")

boothr4_verif(0xdead,0xbeef,17,"")

NameError: name 'int2bin' is not defined

In [472]:
DEBUGMODE = 0;

def fpmul_ifmt(fmt, packa, packb):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Unpack Data
    # Caution: Here we're assuming that all incoming data are in denorm format
    # They have exponents (unlike IEEE terminalogy), but also have an explicit 1
    sgna = packa[0]
    expa = packa[1:exp_len+1]
    mana = [0]*2+packa[exp_len+1:]
    
    sgnb = packb[0]
    expb = packb[1:exp_len+1]
    manb = [0]*2+packb[exp_len+1:]
    if(DEBUGMODE):
        print("A: ", sgna, numdstr(expa), numdstr(mana),"len(mana):",len(mana))
        print("B: ", sgnb, numdstr(expb), numdstr(manb))
    
    # PIPELINE - STEP 1 - Booth Encoder
    # Combinational, decomposes mantissa A into booth recording
    mana_br = boothr4_encode(mana)
    # Find new sign (xor)
    if(sgna==sgnb):
        sgny = 0;
    else:
        sgny = 1;
    
    # PIPELINE - STEP 2~3 - Booth Multiply  (Produce Partial Product)
    # PIPELINE - STEP 4   - Booth Summation (Reuses the adder)
    many = boothr4_slmult(mana_br,manb);
    many = many[4:man_len+4]
    if(DEBUGMODE):
        print(mana_br)
        print(numdstr(many),"len:",len(many))
    
    # PIPELINE - STEP 4   - New Exponent Calculation 
    expy = int2bin(bin2int(expa)+bin2int(expb)-exp_bias+1,exp_len)
    if(DEBUGMODE):
        print(numdstr(expy))
    
    # - PIPELINE STEP 5: Normalize - Shift (Reuses Hardware)    
    # Find the initial one
    many_zerocount = len(many) - len(numdstr(many).lstrip('0'));
    many = (many+[0]*(many_zerocount+1))[many_zerocount:]
    expyn = bin2int(expy)-many_zerocount
    expy = int2bin(expyn,len(expy))
    if(DEBUGMODE):
        print("many_zerocount:", many_zerocount)
        print("after SFT: exp:",numdstr(expy),"many:", numdstr(many));
    
    # Deal with denorm
    if(is_all(expy,0)):
        if(DEBUGMODE):
            print("Denorm")
        return [int(sgny)] + [0]*exp_len + many[0:man_len]
    else:
        a = numdstr([int(sgny)] + expy + many[1:]);
        if(DEBUGMODE):
            print(a[0],a[1:FP32_EXP_LEN+1],a[FP32_EXP_LEN+1:])
        return [int(sgny)] + expy + many[1:]

def fpmul_verif(fpa, fpb, msg):
    fpy = fpa * fpb
    bfpa = fp16_to_ifmt(FMT_FP32N,fpa)
    bfpb = fp16_to_ifmt(FMT_FP32N,fpb)
    if(DEBUGMODE):
        print(numdstr(bfpa))
        print(numdstr(bfpb))
    fpyi = fpmul_ifmt(FMT_FP32N, bfpa, bfpb)
    k = fpdecode(FMT_FP32, fpyi);
    #if(k==fpy):
    #    print('- pass -', msg)
    #else:
    #    print('- FAIL - ', msg)
    print("\n- TEST:",fpa,"x",fpb,"=",fpy)
    print("Expected:  ", fpy)
    print("Actual:    ", k)
    if(fpy!=0):
        print("Error:     ", '{:.5e}'.format(abs(fpy-k)/fpy));
    
fpmul_verif(12345,12,'')
fpmul_verif(1024,0,'')
fpmul_verif(0,768,'')
fpmul_verif(12345e-4,720.01,'')


- TEST: 12345 x 12 = 148140
Expected:   148140
Actual:     148128.0
Error:      8.10045e-05

- TEST: 1024 x 0 = 0
Expected:   0
Actual:     7.450580596923828e-09

- TEST: 0 x 768 = 0
Expected:   0
Actual:     3.725290298461914e-09

- TEST: 1.2345 x 720.01 = 888.8523449999999
Expected:   888.8523449999999
Actual:     888.75
Error:      1.15143e-04


In [10]:
len(boothr4_encode([0]*11))

7

In [28]:
a = strdnum('00010010000101010011010100100100')
b = strdnum('11000000100010010101111010000001')
DEBUGMODE = 1;
y = numdstr(fpadd_ifmt(FMT_FP29i, a, b))

A:  0 001001 000000101010011010100100100
B:  1 100000 000100010010101111010000001
ALU:
A: 000100010010101111010000001
B: 000000000000000000000000000
after ALU: exp: 100000 many: 000100010010101111010000001
many_zerocount: 3
after SFT: exp: 011101 many: 1000100101011110100000010000
1 01110100 0100101011110100000010000


In [27]:
y[1:FP29i_EXP_LEN+1]
y[FP29i_EXP_LEN+1:]

'000100101011110100000010000'